### Imports

In [29]:
import pandas as pd 
import numpy as np
import os
import glob
import folium
import folium.plugins
import datetime
from folium.plugins import HeatMapWithTime
import statistics
import requests

### Area and crime list for user convenience

In [58]:
area_list = [dic['id'].replace('-',' ') for dic in requests.get("https://data.police.uk/api/forces").json()]

In [52]:
crime_list = [dic['name'] for dic in requests.get("https://data.police.uk/api/crime-categories?").json()][1:-1]

### Functions for finding files and creating DF

In [2]:
def find_area_files(area): 
    area = area.replace(" ","-")
    files = glob.glob(f'/Users/andrewsmith/Desktop/Crime_Data_project/data/all_data/**/*{area}-street.csv',recursive = True)
    return files

In [5]:
def create_area_df(area):
    dfs = []
    for file in find_area_files(area): 
        dfs.append(pd.read_csv(file))
    area_df = pd.DataFrame()
    for i in range(len(dfs)): 
        area_df = pd.concat([area_df,dfs[i]]) 
    return area_df
        

### Function for creating a heatmap from user specified area and crime

In [49]:
def heat_map(crime_type,area):
    crime_type = crime_type.capitalize()
    area_df = create_area_df(area)
    individual_crime_df = area_df[area_df["Crime type"]==crime_type][['Longitude','Latitude']]
    individual_crime_df=individual_crime_df.dropna()
    lat_list = individual_crime_df['Latitude'].tolist()
    long_list = individual_crime_df['Longitude'].tolist()
    individual_crime_coordinates = set(zip(lat_list,long_list))
    map_centre = (statistics.mean(lat_list), statistics.mean(long_list))
    base_map = folium.Map(location=map_centre, zoom_start=10)
    
    title_html = '''
             <h3 align="center" style="font-size:20px"><b>{title} in {location} from 6/2018 to 5/2021</b></h3>
             '''.format(title=crime_type.title(),location=area.title())
    base_map.get_root().html.add_child(folium.Element(title_html))
    
                                       
    folium.plugins.HeatMap(individual_crime_coordinates,overlay=True,min_opacity=0.01).add_to(base_map)
    display(base_map)

In [60]:
heat_map('drugs','west yorkshire')

### Function for creating a heatmap with time features from user specified area and crime

In [112]:
def heat_map_time(crime_type,area):
    crime_type = crime_type.capitalize()
    area_df = create_area_df(area)
    crime_area_df= area_df[area_df["Crime type"]==crime_type][['Longitude','Latitude','Month']]
    date_index = list(crime_area_df['Month'].sort_values().unique())
    crime_area_df["Time"] = crime_area_df["Month"].map(lambda x:x.replace('-',''))
    crime_area_df["Time"] = crime_area_df["Time"].map(lambda x:int(x))
    crime_area_df = crime_area_df[["Longitude","Latitude","Time"]]
    crime_area_df = crime_area_df.dropna()
    date_list = list(crime_area_df["Time"].unique())
    
   
    lat_long_list = []
    for i in date_list:
        temp=[]
        for index,instance in crime_area_df[crime_area_df['Time'] == i].iterrows():
            temp.append([instance['Latitude'],instance['Longitude']])
        lat_long_list.append(temp)
    
    map_centre = (statistics.mean(crime_area_df['Latitude'].tolist()),statistics.mean(crime_area_df['Longitude'].tolist()))
    base_map = folium.Map(location=map_centre, zoom_start=10)
    HeatMapWithTime(lat_long_list,auto_play=True,position='bottomright',min_opacity=0.1,index=date_index).add_to(base_map)
    display(base_map)

In [114]:
heat_map_time('drugs','north yorkshire')